In [30]:
import pandas as pd
import pulp as pl
from scipy.optimize import linprog
from pulp import LpMaximize, LpMinimize, LpProblem, LpStatus, lpSum, LpVariable
import re

In [31]:
routes = pd.read_csv('outputs/selected_routes.csv')
attractions = pd.read_csv('mr-qap/mr-qap-results.csv').round(2)

In [32]:
def adjust_duration_transit(row):
    """
    Hardcoded adjustment to transit (divide duration by 4) as an equivalency measure to driving.
    In that case, a 8 hour train ride is equivalent to a 2 hour car ride.
    """
    if row['mode'] == 'transit':
        row.duration = row.duration / 4
    return row

routes = routes.apply(lambda row: adjust_duration_transit(row), axis=1)

In [33]:
attractions = dict(zip(attractions.country, attractions.predicted_shares))

In [34]:
attractions

{'Hungary': 0.11,
 'Belarus': 0.03,
 'Moldova': 0.12,
 'Poland': 0.18,
 'Romania': 0.17,
 'Russian Federation': 0.19,
 'Slovakia': 0.2}

`min{sum(duration)}`

each `(conflict, crossing)` pair is a variable for total fatalities routed

the sum of `sum{(conflict, crossing)}` for each conflict `i` must equal total fatalities for that conflict.

`(conflict, crossing)` is continuous but can equal 0

the sum of each `country` fatalities divided by total fatalities must equal that country's `predicted_shares`

In [35]:
def get_fatalities(conflict):
    fatalities = routes[routes.conflict==conflict].iloc[0].fatalities
    return fatalities

In [36]:
reg = re.compile(r'[\W]')

In [37]:
def strip_text(text):
    return re.sub(reg, '', text).lower()

In [38]:
conflicts = routes.conflict.unique()
crossings = routes.crossing.unique()
countries = routes.crossing_country.unique()

In [39]:
model = LpProblem(name="refugee-routing", sense=LpMinimize)

In [40]:
variables = {}
variables_lookup = {}

for conf in conflicts:
    variables[conf] = []
    
for kk, vv in routes.iterrows():
    conf = strip_text(vv.conflict)
    cross = strip_text(vv.crossing)
    country = strip_text(vv.crossing_country)
    mode = vv['mode']
    v = f"{conf}__{cross}__{country}__{mode}"
    x = LpVariable(name=v, lowBound=0, cat='Integer')
    variables[vv.conflict].append(x)
    
    variables_lookup[v] = dict(conflict=vv.conflict,
                               crossing=vv.crossing,
                               country=vv.crossing_country,
                               mode=mode,
                               duration=vv.duration)

In [41]:
vars_by_country = {}
for country in countries:
    vars_by_country[country] = []

for kk, vv in variables.items():
    for i in vv:
        country = variables_lookup[i.name]['country']
        vars_by_country[country].append(i)

In [42]:
for conf in conflicts:
    model += (lpSum(variables[conf]) == get_fatalities(conf), 
              f"{conf}_fatalities_constraint") 

In [43]:
total_fatalities = routes.drop_duplicates('conflict').fatalities.sum()

In [44]:
for c in countries:
    country_vars = vars_by_country[c]
        
    model += (lpSum(country_vars)/total_fatalities == attractions[c],
              f"{c}_attraction")

In [45]:
obj_func_array = []
for kk, vv in variables.items():
    for i in vv:
        fatalities = i
        duration = variables_lookup[i.name]['duration']
        obj_func_array.append(fatalities*duration)

In [46]:
model += lpSum(obj_func_array)

In [48]:
status = model.solve()

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/brandonrose/opt/anaconda3/envs/b39/lib/python3.9/site-packages/pulp/apis/../solverdir/cbc/osx/64/cbc /var/folders/pp/vgfp1wf143q46m_8v6qbt3bw0000gn/T/0e60f939e3c2480da2168513608de710-pulp.mps timeMode elapsed branch printingOptions all solution /var/folders/pp/vgfp1wf143q46m_8v6qbt3bw0000gn/T/0e60f939e3c2480da2168513608de710-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 58 COLUMNS
At line 2089 RHS
At line 2143 BOUNDS
At line 2550 ENDATA
Problem MODEL has 53 rows, 406 columns and 812 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 1.60807e+08 - 0.00 seconds
Cgl0004I processed model has 53 rows, 322 columns (322 integer (0 of which binary)) and 644 elements
Cbc0013I At root node, 10 cuts changed objective from 1.6081083e+08 to 1.6081083e+08 in 1 passes
Cbc0014I Cut generator 0 (Probi

In [49]:
print(f"status: {model.status}, {LpStatus[model.status]}")
print(f"objective: {model.objective.value()}")


for var in model.variables():
    print(f"{var.name}: {var.value()}")

for name, constraint in model.constraints.items():
    print(f"{name}: {constraint.value()}")

status: -1, Infeasible
objective: 160806992.54999998
baryshivka__choptysazakhon__hungary__driving: 0.0
baryshivka__malyibereznyiubla__slovakia__driving: 0.0
baryshivka__platonovehoianulnou__moldova__driving: 0.0
baryshivka__porubnesiret__romania__driving: 0.0
baryshivka__senkivkanoviyurkovychi__russianfederation__driving: 0.0
baryshivka__slavutychkomaryn__belarus__driving: 0.0
baryshivka__yahodyndorohusk__poland__driving: 40.0
bilohorivka__choptysazakhon__hungary__driving: 0.0
bilohorivka__krasnatalivkavoloshino__russianfederation__driving: 0.0
bilohorivka__malyibereznyiubla__slovakia__driving: 0.0
bilohorivka__platonovehoianulnou__moldova__driving: 0.0
bilohorivka__porubnesiret__romania__driving: 459.84
bilohorivka__vilchaoleksandrivka__belarus__driving: 0.0
bilohorivka__yahodyndorohusk__poland__driving: 165.16
borodianka__choptysazakhon__hungary__driving: 0.0
borodianka__malyibereznyiubla__slovakia__driving: 0.0
borodianka__mohylivpodilskyiotach__moldova__driving: 0.0
borodianka__por

In [50]:
refugee_counts = {}
for c in countries:
    refugee_counts[c] = 0

In [51]:
for var in model.variables():
    for c in countries:
        if c.split(' ')[0].lower() in var.name.lower():
            refugee_counts[c] += var.value()

In [52]:
for kk, vv in refugee_counts.items():
    ratio = vv/total_fatalities
    print(f"{kk}: {ratio}")

Poland: 0.18
Moldova: 0.12
Romania: 0.16999999999999998
Slovakia: 0.2
Hungary: 0.10999999999999999
Belarus: 0.03
Russian Federation: 0.19


In [53]:
attractions

{'Hungary': 0.11,
 'Belarus': 0.03,
 'Moldova': 0.12,
 'Poland': 0.18,
 'Romania': 0.17,
 'Russian Federation': 0.19,
 'Slovakia': 0.2}

In [54]:
results = []
for var in model.variables():
    results.append((var, var.value()))
results = pd.DataFrame(results, columns=['variable','value'])    

In [55]:
results['conflict'] = results.variable.apply(lambda x: variables_lookup[x.name]['conflict'])
results['crossing'] = results.variable.apply(lambda x: variables_lookup[x.name]['crossing'])
results['country'] = results.variable.apply(lambda x: variables_lookup[x.name]['country'])

In [56]:
results.to_csv('outputs/linear_prog_results.csv', index=False)